# Hands-on Tutorial on XAI (Introduction)
### Machine Learning and Dynamical Systems Seminar hosted by the Alan Turing Institute

[Link](https://github.com/steveazzolin/gdl_tutorial_turinginst) to online material

![](./SCM_XAITutorial.png)

Given the Structural Causal Model defined above, we will sample some data from the underlying distribution and train two ML models with the goal of inspecting their behaviour under environment shifts. Then, by inspecting the learned weights, we try to judge whether the model learned the expected decision patterns

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [2]:
def create_data_env(n_samples, mu_1, dev_2, mu_3):
  x1 = np.random.randn(n_samples, 1) + mu_1
  y = x1 >= mu_1
  x2 = y + np.random.randn(n_samples, 1) * dev_2
  x3 = np.random.randn(n_samples, 1) + mu_3

  X = np.concatenate([x1,x2,x3], axis=-1)
  y = np.squeeze(y)
  return X, y

Create data from 3 different environments

In [9]:
n_samples = 1000
mu_1 = 5
mu_3 = 2.5
dev_2_env_1 = 0.1
dev_2_env_2 = 1
dev_2_env_3 = 10

X, y = {}, {}

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_1, mu_3)
X["env_1"] = X_env
y["env_1"] = y_env

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_2, mu_3)
X["env_2"] = X_env
y["env_2"] = y_env

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_3, mu_3)
X["env_3"] = X_env
y["env_3"] = y_env


print(X["env_1"].shape, y["env_1"].shape)

(1000, 3) (1000,)


Fit a Simple **LogisticRegression** on the first environment

In [10]:
clf = LogisticRegression(random_state=42).fit(X["env_1"], y["env_1"])
preds = clf.predict(X["env_1"])
acc = clf.score(X["env_1"], y["env_1"])
print("Accuracy of the model on env 1 = ", acc)

Accuracy of the model on env 1 =  1.0


Does it generalize to the other environments?

In [12]:
acc = clf.score(X["env_2"], y["env_2"])
print("Accuracy of the model on env 2 = ", acc)

acc = clf.score(X["env_3"], y["env_3"])
print("Accuracy of the model on env 3 = ", acc)

Accuracy of the model on env 2 =  0.823
Accuracy of the model on env 3 =  0.547


Why? Can we inspect what it has learned?
The **LogisticRegression** is defined as:

$\hat{y} = \sigma(w_1x_1 + w_2x_2 + w_3x_3$)

So we can easily inspect the individual weights

In [6]:
clf.coef_

array([[ 3.38568565,  5.88345797, -0.01985111]])

So, the model gave a large importance to $x_2$ even tough is not causally asssociated to $y$. This opens the model to several vulnerabilities associated to distribution/environment shifts.

Let's try with a Deep Learning model, specifically a **MLP** with 2 layers and 100 hidden units

In [7]:
clf = MLPClassifier(random_state=42, max_iter=700).fit(X["env_1"], y["env_1"])
for e in ["env_1", "env_2", "env_3"]:
  acc = clf.score(X[e], y[e])
  print(f"Accuracy of the model on {e}= ", acc)

Accuracy of the model on env_1=  1.0
Accuracy of the model on env_2=  0.713
Accuracy of the model on env_3=  0.547


Let's inspect again the weights

In [8]:
Ws = clf.coefs_
print("Layers of the network: ", len(Ws))
print("Num of params of first layer: ", Ws[0].shape)
print(Ws[0])

Layers of the network:  2
Num of params of first layer:  (3, 100)
[[-4.16680079e-10  2.65819350e-01  1.33165195e-01  7.08711959e-02
  -1.50842059e-02 -9.99475943e-02  3.75521525e-02  1.59997649e-01
  -3.10241391e-02  1.46202248e-01 -6.50734781e-04  2.45924444e-01
   1.90099158e-01 -1.00606615e-01 -1.72595427e-01 -3.32581201e-03
  -1.15235761e-07  4.29178214e-02  9.36066009e-02 -1.31721312e-01
   8.93770802e-02 -6.24034204e-02  3.19306382e-02 -9.76831014e-10
   3.82997391e-02  2.21005562e-01 -1.07960862e-01  9.30508792e-02
   2.48610796e-02 -4.28500690e-04  2.43072335e-02 -7.61906644e-02
  -3.09254789e-04  1.92429421e-01  2.14695486e-01  1.79954410e-01
   4.51944474e-03  1.60015401e-02  1.16094101e-01  1.21072248e-14
  -2.18219138e-01  3.23108410e-02 -2.42251354e-02  1.55037236e-01
  -1.21314975e-06  9.14858644e-02 -4.49406049e-02  1.00819739e-01
   1.69624948e-02 -1.92149866e-02  2.59761115e-01  9.92340312e-02
   1.66180228e-01  2.27353889e-01  1.02659080e-02  1.20733002e-01
  -5.05345

With this deep learning model, even if failry shallow, we can't assess the goodness of the model by inspewcting the parameters, since theiy are simply too many.